# Data Preprocessing

#### Import libraries and load our data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
#Let's load our datasets
weather = pd.read_csv('../CapstoneTwo/weather_clean.csv')
train = pd.read_csv('../CapstoneTwo/train_clean.csv')
test = pd.read_csv('../CapstoneTwo/test_clean.csv')

#### Create functions to handle feature engineering

In [4]:
#Let's create functions to handle datetime
def datetime(df):
    df['Date'] = pd.to_datetime(df['Date'])

#And extract the year, month, week, and day
def ymwd(df):
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month
    df['Week'] = df['Date'].dt.isocalendar().week
    df['Day'] = df['Date'].dt.day_of_year

In [5]:
#A function to shift weather features
def lag_weather(df):

    df['Tmax_1w'] = df['Tmax'].rolling(window =7).mean()
    df['Tmax_2w'] = df['Tmax'].rolling(window =14).mean()
    df['Tmax_3w'] = df['Tmax'].rolling(window =21).mean()

    df['Tmin_1w'] = df['Tmin'].rolling(window =7).mean()
    df['Tmin_2w'] = df['Tmin'].rolling(window =14).mean()
    df['Tmin_3w'] = df['Tmin'].rolling(window =21).mean()

    df['Tavg_1w'] = df['Tavg'].rolling(window =7).mean()
    df['Tavg_2w'] = df['Tavg'].rolling(window =14).mean()
    df['Tavg_3w'] = df['Tavg'].rolling(window =21).mean()

    df['Depart_1w'] = df['Depart'].rolling(window =7).mean()
    df['Depart_2w'] = df['Depart'].rolling(window =14).mean()
    df['Depart_3w'] = df['Depart'].rolling(window =21).mean()

    df['DewPoint_1w'] = df['DewPoint'].rolling(window =7).mean()
    df['DewPoint_2w'] = df['DewPoint'].rolling(window =14).mean()
    df['DewPoint_3w'] = df['DewPoint'].rolling(window =21).mean()

    df['WetBulb_1w'] = df['WetBulb'].rolling(window =7).mean()
    df['WetBulb_2w'] = df['WetBulb'].rolling(window =14).mean()
    df['WetBulb_3w'] = df['WetBulb'].rolling(window =21).mean()

    df['Heat_1w'] = df['Heat'].rolling(window =7).mean()
    df['Heat_2w'] = df['Heat'].rolling(window =14).mean()
    df['Heat_3w'] = df['Heat'].rolling(window =21).mean()

    df['Cool_1w'] = df['Cool'].rolling(window =7).mean()
    df['Cool_2w'] = df['Cool'].rolling(window =14).mean()
    df['Cool_3w'] = df['Cool'].rolling(window =21).mean()

    df['PrecipTotal_1w'] = df['PrecipTotal'].rolling(window =7).mean()
    df['PrecipTotal_2w'] = df['PrecipTotal'].rolling(window =14).mean()
    df['PrecipTotal_3w'] = df['PrecipTotal'].rolling(window =21).mean()

    df['StnPressure_1w'] = df['StnPressure'].rolling(window =7).mean()
    df['StnPressure_2w'] = df['StnPressure'].rolling(window =14).mean()
    df['StnPressure_3w'] = df['StnPressure'].rolling(window =21).mean()

    df['SeaLevel_1w'] = df['SeaLevel'].rolling(window =7).mean()
    df['SeaLevel_2w'] = df['SeaLevel'].rolling(window =14).mean()
    df['SeaLevel_3w'] = df['SeaLevel'].rolling(window =21).mean()

    df['ResultSpeed_1w'] = df['ResultSpeed'].rolling(window =7).mean()
    df['ResultSpeed_2w'] = df['ResultSpeed'].rolling(window =14).mean()
    df['ResultSpeed_3w'] = df['ResultSpeed'].rolling(window =21).mean()

    df['ResultDir_1w'] = df['ResultDir'].rolling(window =7).mean()
    df['ResultDir_2w'] = df['ResultDir'].rolling(window =14).mean()
    df['ResultDir_3w'] = df['ResultDir'].rolling(window =21).mean()

    df['AvgSpeed_1w'] = df['AvgSpeed'].rolling(window =7).mean()
    df['AvgSpeed_2w'] = df['AvgSpeed'].rolling(window =14).mean()
    df['AvgSpeed_3w'] = df['AvgSpeed'].rolling(window =21).mean()

    return df

In [6]:
#Let's create a function to get dummy variables for species
def spec_dummies(df):
    df = pd.concat([df, pd.get_dummies(df['Species'], drop_first=True)], axis=1)
# We can drop the species column now that we have our dummies.
    df.drop('Species', axis=1, inplace=True)
    return df

In [7]:
#Let's create a function to cast features to float
def float(df):
    for col in df.columns:
        try:
            df[col] = df[col].astype(float)
        except:
            print(col, 'Cannot be transformed into a float')
            pass
    return df

In [8]:
#We'll deal with date features in all of the datasets now
datetime(weather)
datetime(train)
datetime(test)

ymwd(weather)
ymwd(train)
ymwd(test)

#### Let's work on feature engineering for our weather dataset

In [9]:
#Let's lag features in weather using our predefined function
weather = lag_weather(weather)

In [10]:
#We need to handle the missing values created from shifting features
weather.interpolate(method ='bfill', limit_direction ='backward', inplace=True)

#weather.fillna(0)

#### Let's create dummy variables for species in our train and test set.

In [11]:
train = spec_dummies(train)
test = spec_dummies(test)

In [12]:
#Combine datasets
train_final = pd.merge(train, weather,on=['Date', 'Year', 'Month', 'Week', 'Day'],how='left')
test_final = pd.merge(test, weather,on=['Date', 'Year', 'Month', 'Week', 'Day'],how='left')

In [13]:
#Drop additional columns we may not need
train_final.drop(['Sunrise', 'Sunset'], axis=1, inplace=True)

In [14]:
#Let's save our combined datasets
train_final.to_csv('../CapstoneTwo/train_final.csv',index=False)
test_final.to_csv('../CapstoneTwo/test_final.csv',index=False)


In [15]:
train_final.head()

,Date,Latitude,Longitude,WnvPresent,Year,Month,Week,Day,CULEX PIPIENS,CULEX PIPIENS/RESTUANS,CULEX RESTUANS,CULEX SALINARIUS,CULEX TARSALIS,CULEX TERRITANS,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,Tmax_1w,Tmax_2w,Tmax_3w,Tmin_1w,Tmin_2w,Tmin_3w,Tavg_1w,Tavg_2w,Tavg_3w,Depart_1w,Depart_2w,Depart_3w,DewPoint_1w,DewPoint_2w,DewPoint_3w,WetBulb_1w,WetBulb_2w,WetBulb_3w,Heat_1w,Heat_2w,Heat_3w,Cool_1w,Cool_2w,Cool_3w,PrecipTotal_1w,PrecipTotal_2w,PrecipTotal_3w,StnPressure_1w,StnPressure_2w,StnPressure_3w,SeaLevel_1w,SeaLevel_2w,SeaLevel_3w,ResultSpeed_1w,ResultSpeed_2w,ResultSpeed_3w,ResultDir_1w,ResultDir_2w,ResultDir_3w,AvgSpeed_1w,AvgSpeed_2w,AvgSpeed_3w
0,2007-05-29,41.954690,-87.800991,0,2007,5,22,149,0,1,0,0,0,0,88.0,62.5,75.25,10.0,58.5,65.5,0.0,10.5,0.0,29.415,30.1,5.8,17.0,6.95,79.0,76.107143,76.261905,58.0,52.75,52.833333,68.5,64.428571,64.547619,5.714286,2.928571,4.47619,50.928571,45.357143,45.738095,58.857143,54.357143,54.619048,1.285714,3.571429,3.428571,5.0,3.25,3.190476,0.110714,0.069286,0.073095,29.435714,29.434286,29.399048,30.120714,30.121429,30.087143,7.421429,8.021429,8.17619,15.857143,16.214286,15.666667,9.328571,9.682143,9.845238
1,2007-05-29,41.954690,-87.800991,0,2007,5,22,149,0,0,1,0,0,0,88.0,62.5,75.25,10.0,58.5,65.5,0.0,10.5,0.0,29.415,30.1,5.8,17.0,6.95,79.0,76.107143,76.261905,58.0,52.75,52.833333,68.5,64.428571,64.547619,5.714286,2.928571,4.47619,50.928571,45.357143,45.738095,58.857143,54.357143,54.619048,1.285714,3.571429,3.428571,5.0,3.25,3.190476,0.110714,0.069286,0.073095,29.435714,29.434286,29.399048,30.120714,30.121429,30.087143,7.421429,8.021429,8.17619,15.857143,16.214286,15.666667,9.328571,9.682143,9.845238
2,2007-05-29,41.994991,-87.769279,0,2007,5,22,149,0,0,1,0,0,0,88.0,62.5,75.25,10.0,58.5,65.5,0.0,10.5,0.0,29.415,30.1,5.8,17.0,6.95,79.0,76.107143,76.261905,58.0,52.75,52.833333,68.5,64.428571,64.547619,5.714286,2.928571,4.47619,50.928571,45.357143,45.738095,58.857143,54.357143,54.619048,1.285714,3.571429,3.428571,5.0,3.25,3.190476,0.110714,0.069286,0.073095,29.435714,29.434286,29.399048,30.120714,30.121429,30.087143,7.421429,8.021429,8.17619,15.857143,16.214286,15.666667,9.328571,9.682143,9.845238
3,2007-05-29,41.974089,-87.824812,0,2007,5,22,149,0,1,0,0,0,0,88.0,62.5,75.25,10.0,58.5,65.5,0.0,10.5,0.0,29.415,30.1,5.8,17.0,6.95,79.0,76.107143,76.261905,58.0,52.75,52.833333,68.5,64.428571,64.547619,5.714286,2.928571,4.47619,50.928571,45.357143,45.738095,58.857143,54.357143,54.619048,1.285714,3.571429,3.428571,5.0,3.25,3.190476,0.110714,0.069286,0.073095,29.435714,29.434286,29.399048,30.120714,30.121429,30.087143,7.421429,8.021429,8.17619,15.857143,16.214286,15.666667,9.328571,9.682143,9.845238
4,2007-05-29,41.974089,-87.824812,0,2007,5,22,149,0,0,1,0,0,0,88.0,62.5,75.25,10.0,58.5,65.5,0.0,10.5,0.0,29.415,30.1,5.8,17.0,6.95,79.0,76.107143,76.261905,58.0,52.75,52.833333,68.5,64.428571,64.547619,5.714286,2.928571,4.47619,50.928571,45.357143,45.738095,58.857143,54.357143,54.619048,1.285714,3.571429,3.428571,5.0,3.25,3.190476,0.110714,0.069286,0.073095,29.435714,29.434286,29.399048,30.120714,30.121429,30.087143,7.421429,8.021429,8.17619,15.857143,16.214286,15.666667,9.328571,9.682143,9.845238


In [16]:
test_final.head()

,Date,Latitude,Longitude,Year,Month,Week,Day,CULEX PIPIENS,CULEX PIPIENS/RESTUANS,CULEX RESTUANS,CULEX SALINARIUS,CULEX TARSALIS,CULEX TERRITANS,UNSPECIFIED CULEX,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,Tmax_1w,Tmax_2w,Tmax_3w,Tmin_1w,Tmin_2w,Tmin_3w,Tavg_1w,Tavg_2w,Tavg_3w,Depart_1w,Depart_2w,Depart_3w,DewPoint_1w,DewPoint_2w,DewPoint_3w,WetBulb_1w,WetBulb_2w,WetBulb_3w,Heat_1w,Heat_2w,Heat_3w,Cool_1w,Cool_2w,Cool_3w,PrecipTotal_1w,PrecipTotal_2w,PrecipTotal_3w,StnPressure_1w,StnPressure_2w,StnPressure_3w,SeaLevel_1w,SeaLevel_2w,SeaLevel_3w,ResultSpeed_1w,ResultSpeed_2w,ResultSpeed_3w,ResultDir_1w,ResultDir_2w,ResultDir_3w,AvgSpeed_1w,AvgSpeed_2w,AvgSpeed_3w
0,2008-06-11,41.95469,-87.800991,2008,6,24,163,0,1,0,0,0,0,0,86.0,63.5,74.75,7.0,55.5,64.0,0.0,10.0,416.0,1926.0,0.0,29.31,29.98,9.15,18.0,10.2,83.428571,79.642857,74.833333,65.714286,60.357143,55.47619,74.571429,70.0,65.154762,8.0,4.357143,0.47619,64.571429,58.607143,53.095238,68.428571,63.392857,58.738095,0.0,0.821429,3.738095,9.785714,6.071429,4.142857,0.285714,0.223929,0.16119,29.162143,29.150714,29.228571,29.826429,29.832143,29.910952,10.657143,8.160714,8.840476,20.428571,16.392857,14.071429,12.5,9.839286,10.378571
1,2008-06-11,41.95469,-87.800991,2008,6,24,163,0,0,1,0,0,0,0,86.0,63.5,74.75,7.0,55.5,64.0,0.0,10.0,416.0,1926.0,0.0,29.31,29.98,9.15,18.0,10.2,83.428571,79.642857,74.833333,65.714286,60.357143,55.47619,74.571429,70.0,65.154762,8.0,4.357143,0.47619,64.571429,58.607143,53.095238,68.428571,63.392857,58.738095,0.0,0.821429,3.738095,9.785714,6.071429,4.142857,0.285714,0.223929,0.16119,29.162143,29.150714,29.228571,29.826429,29.832143,29.910952,10.657143,8.160714,8.840476,20.428571,16.392857,14.071429,12.5,9.839286,10.378571
2,2008-06-11,41.95469,-87.800991,2008,6,24,163,1,0,0,0,0,0,0,86.0,63.5,74.75,7.0,55.5,64.0,0.0,10.0,416.0,1926.0,0.0,29.31,29.98,9.15,18.0,10.2,83.428571,79.642857,74.833333,65.714286,60.357143,55.47619,74.571429,70.0,65.154762,8.0,4.357143,0.47619,64.571429,58.607143,53.095238,68.428571,63.392857,58.738095,0.0,0.821429,3.738095,9.785714,6.071429,4.142857,0.285714,0.223929,0.16119,29.162143,29.150714,29.228571,29.826429,29.832143,29.910952,10.657143,8.160714,8.840476,20.428571,16.392857,14.071429,12.5,9.839286,10.378571
3,2008-06-11,41.95469,-87.800991,2008,6,24,163,0,0,0,1,0,0,0,86.0,63.5,74.75,7.0,55.5,64.0,0.0,10.0,416.0,1926.0,0.0,29.31,29.98,9.15,18.0,10.2,83.428571,79.642857,74.833333,65.714286,60.357143,55.47619,74.571429,70.0,65.154762,8.0,4.357143,0.47619,64.571429,58.607143,53.095238,68.428571,63.392857,58.738095,0.0,0.821429,3.738095,9.785714,6.071429,4.142857,0.285714,0.223929,0.16119,29.162143,29.150714,29.228571,29.826429,29.832143,29.910952,10.657143,8.160714,8.840476,20.428571,16.392857,14.071429,12.5,9.839286,10.378571
4,2008-06-11,41.95469,-87.800991,2008,6,24,163,0,0,0,0,0,1,0,86.0,63.5,74.75,7.0,55.5,64.0,0.0,10.0,416.0,1926.0,0.0,29.31,29.98,9.15,18.0,10.2,83.428571,79.642857,74.833333,65.714286,60.357143,55.47619,74.571429,70.0,65.154762,8.0,4.357143,0.47619,64.571429,58.607143,53.095238,68.428571,63.392857,58.738095,0.0,0.821429,3.738095,9.785714,6.071429,4.142857,0.285714,0.223929,0.16119,29.162143,29.150714,29.228571,29.826429,29.832143,29.910952,10.657143,8.160714,8.840476,20.428571,16.392857,14.071429,12.5,9.839286,10.378571
